In [1]:
import pandas as pd
import numpy as np
import pickle 
from tqdm import tqdm 
from collections import Counter,OrderedDict
from sklearn.metrics.pairwise import cosine_similarity
import nltk 
from nltk.corpus import stopwords
import string  

nltk.download("stopwords")

def flatten_comprehension(matrix):
    #https://realpython.com/python-flatten-list/
    return [item for row in matrix for item in row]

# Load data
behaviors = pd.read_csv('./small_training_data/behaviors.tsv', delimiter='\t', header=None)
news = pd.read_csv('./small_training_data/news.tsv', delimiter='\t', header=None)

# Naming columns
behaviors.columns = ["impression_id", "user_id", "time", "history", "impressions"]
news.columns = ["news_id", "category", "subcategory", "title", "abstract", "url", "title_entities", "abstract_entities"]


# Extracting clicked news from behaviors, this is a column of lists of the clicked news (tagget with 1) for each impression
behaviors['clicked_news'] = behaviors['impressions'].apply(lambda x: [imp.split('-')[0] for imp in x.split() if imp.split('-')[1] == '1'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/hal9000/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#define unique users and their read articles
unique_users_series = behaviors["user_id"].drop_duplicates()
unique_users_dict = {}  #user_id coupled to news_articles

#aggregate all read articles and couple it to the user in the unique_users_frame
i = 0
for user_id in tqdm(unique_users_series,ascii=True):
    user_specific_behavior = behaviors[behaviors["user_id"] == user_id]
    try:
        history_list = flatten_comprehension([elem.split(" ") for elem in user_specific_behavior["history"]])
    except:
        pass 
    clicked_list = flatten_comprehension([elem for elem in user_specific_behavior["clicked_news"]])
    all_read_articles = history_list + clicked_list
    unique_users_dict[user_id] = all_read_articles


with open('unique_users.pkl', 'wb') as file: #serialize so we don't have to do this inefficient code again
    pickle.dump(unique_users_dict,file)



100%|####################################################################################################################| 50000/50000 [03:52<00:00, 215.03it/s]


In [38]:
#define the embedding. (what words we look for)

def remove_punctuation(sentence:str) -> str:
    sentence = sentence.replace("'","")
    for elem in string.punctuation:
        sentence = sentence.replace(elem," ")
    return sentence

def remove_stopwords(sentence:str)-> str:
    stop_words = list(set(stopwords.words("english")))
    sentence= [ elem for elem in sentence.split(" ") if elem not in stop_words]
        
    return " ".join(sentence)

def preprocess(sentence:str) -> Counter:
    counter = Counter(remove_punctuation(remove_stopwords(sentence.lower())).split(" "))
    counter[''] =0 
    return counter


# total_title = ""
# for title in tqdm(news["title"],ascii=True):
#     total_title += " "+title 
# total_BOW = preprocess(total_title)


# sorted_BOW = sorted(OrderedDict(total_BOW).items(),key=lambda x: x[1],reverse=True)
# print(sorted_BOW)
    

In [39]:
# embedding_words = [elem[0] for elem in sorted_BOW[:1000]]

with open('unique_users.pkl', 'rb') as file:
    users = pickle.load(file)

user_BOW = {} #{"user_id":embedding,etc.}

#compute user embeddings
for user in tqdm(users,ascii=True):
    news_ids = users[user]    
    counter =preprocess(news["title"][news["news_id"].isin(news_ids)].map(str.lower).apply(lambda x: x +" ").agg("sum"))
    
    user_BOW[user] = counter
    
with open('userBOW.pkl', 'wb') as file: #serialize so we don't have to do this inefficient code again
    pickle.dump(user_BOW,file)
    

100%|####################################################################################################################| 50000/50000 [01:28<00:00, 567.06it/s]


In [37]:
with open('userBOW.pkl', 'rb') as file:
    user_BOW = pickle.load(file)

df = pd.DataFrame({"user1":[],"user2":[],"similarity":[]})

for user1 in tqdm(user_BOW,ascii=True):
    for user2 in user_BOW:
        print((user_BOW[user1] & user_BOW[user2]).total())
        similarity = (user_BOW[user1] - user_BOW[user2]).total()        
        df_row = pd.DataFrame({"user1":[user1],"user2":[user2],"similarity":[1]})
        print(pd.concat([df, df_row], join="inner")) 

        break
    break 
    



  0%|                                                                                                                                 | 0/50000 [00:00<?, ?it/s]

Counter({'former': 2, 'dead': 2, 'flight': 2, 'michigan': 2, 'state': 2, 'joe': 1, 'biden': 1, 'reportedly': 1, 'denied': 1, 'communion': 1, 'south': 1, 'carolina': 1, 'church': 1, 'stance': 1, 'abortion': 1, 'wedding': 1, 'photo': 1, 'canine': 1, 'best': 1, 'man': 1, 'captures': 1, 'deep': 1, 'dogs': 1, 'love': 1, 'truly': 1, 'wheel': 1, 'fortune': 1, 'guest': 1, 'delivers': 1, 'hilarious': 1, 'rails': 1, 'introduction': 1, 'three': 1, 'takeaways': 1, 'yankees': 1, 'alcs': 1, 'game': 1, '5': 1, 'victory': 1, 'astros': 1, 'us': 1, 'senator': 1, 'kay': 1, 'hagan': 1, '66': 1, 'robert': 1, 'evans': 1, 'chinatown': 1, 'producer': 1, 'paramount': 1, 'chief': 1, 'dies': 1, '89': 1, 'rosie': 1, 'odonnell': 1, 'barbara': 1, 'walters': 1, 'up': 1, 'speaking': 1, 'people': 1, 'right': 1, 'four': 1, 'attendants': 1, 'arrested': 1, 'miamis': 1, 'airport': 1, 'bringing': 1, 'thousands': 1, 'cash': 1, 'police': 1, 'say': 1, 'sends': 1, 'breakup': 1, 'tweet': 1, 'notre': 1, 'dame': 1, 'series': 1, '

In [ ]:
"""
NOTE 

instead of cosine similarity we use Jaccard similarity J (https://stats.stackexchange.com/questions/289400/quantify-the-similarity-of-bags-of-words)
because the BOW embeddings are really sparse except if you use to common words ( "a", "is", "of",etc) 
This would be counter intuitive to use. 

"""